In [ ]:
import pandas

print("Importing names... ")
names = pandas.read_csv("data/names.csv", usecols=["name", "n_publs", "likely_gender", "score"])
print("Names imported.")

In [ ]:
# Setting index & accessing cells: https://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
names = names.set_index("name", drop = False)
print(names)

In [ ]:
print("Getting private key... ")
# Get private API Key for NamSor API v2 (contained in txt file)
key = ''

# Import personal key
with open("key.txt", "r") as file:
    key = file.read()

if(len(key) > 0):
    print("Got private key.")
else: 
    print("Could not find private key. Please check the file name and make sure you have an API key.")

In [ ]:
# Trying out NamSor API v2 to get the gender of a name
# https://www.namsor.com/
# https://v2.namsor.com/NamSorAPIv2/apidoc.html
# using NamSor API v2 Python SDK
# https://github.com/namsor/namsor-python-sdk2
# licensed under GNU Affero General Public License v3.0

# Alternatives? https://genderize.io/ -> But only first name!

# Following script taken from https://github.com/namsor/namsor-python-sdk2 "Getting Started" 
# and adapted to keep key private and remove unnecessary lines.
# It tests the connection to the NamSor API

print("Setting up NamSor API v2 connection settings...")

import openapi_client
from openapi_client.rest import ApiException

# Configure API key authorization: api_key
configuration = openapi_client.Configuration()
configuration.api_key['X-API-KEY'] = key
# create an instance of the API class
pers_api_instance = openapi_client.PersonalApi(openapi_client.ApiClient(configuration))

In [ ]:
# Formatting the names using the API's models
def createPersonalNameIn(name):
    return openapi_client.PersonalNameIn(id=name, name=name)

list_of_names = list(map(createPersonalNameIn, names.index.values))

In [ ]:
print("The formatted names look like that: {}".format(list_of_names[:100]))
'''
To classify by hand: [
{'id': '(David) Jing Dai', 'name': '(David) Jing Dai'}, 
{'id': '(Max) Zong-Ming Cheng', 'name': '(Max) Zong-Ming Cheng'}, 
{'id': '(Sophy) Shu-Jiun Chen', 'name': '(Sophy) Shu-Jiun Chen'}, 
{'id': '(Zhou) Bryan Bai', 'name': '(Zhou) Bryan Bai'}]
'''

In [ ]:
batch_personal_name_in = openapi_client.BatchPersonalNameIn(personal_names=list_of_names)

In [ ]:
len(list_of_names) 

In [ ]:
# Now testing the repeated calling of the API, cycling through a list of names, sending in
# one batch at a time and saving the result answer by answer.

batch_size = 1000 #1000 is the API limit given by NamSor
start = 0
end = batch_size
result = []

names_stack = list(names.index.values)

list_of_names = list(map(createPersonalNameIn, names.index.values))

while (len(names_stack) >= batch_size):
    try:
        current_batch = list(map(createPersonalNameIn, names_stack[start:end]))
        batch_personal_name_in = openapi_client.BatchPersonalNameIn(personal_names=current_batch)
        api_response = pers_api_instance.gender_full_batch(batch_personal_name_in=batch_personal_name_in)
        result = result + api_response.personal_names
        
        del names_stack[start:end]
        
        # get remaining names if they are less than a batch size
        if(len(names_stack) < batch_size and len(names_stack) > 0):
            current_batch = list(map(createPersonalNameIn, names_stack))
            batch_personal_name_in = openapi_client.BatchPersonalNameIn(personal_names=current_batch)
            api_response = pers_api_instance.gender_full_batch(batch_personal_name_in=batch_personal_name_in)
            result = result + api_response.personal_names
            names_stack = []
        
        print("Batch of names analyzed")
    except ApiException as e:
        print("Exception when calling PersonalApi: gender_full_batch: %s\n" % e)


print("All batches analyzed.")

In [ ]:
# Convert results (list of openapi_client.models.personal_name_gendered_out.PersonalNameGenderedOut) to (list of dictionaries)

for oapi_el in result:
    names.at[oapi_el.name, 'likely_gender'] = oapi_el.likely_gender
    names.at[oapi_el.name, 'score'] = oapi_el.score

In [ ]:
names